# Odds, stadium info, weather
**This notebook will create CSVs that give us the following variables:**<br>
<br>
**home_team:** Binary indicator of home team<br>
**outdoors:** Binary indicating if game will be played outdoors<br>
**grass:** Binary indicating if game will be played on grass<br>
**spread_line:** Point spread reversed so that if a team is favored, number is positive. If team is underdog, number is negative.<br>
**total_line:** Predicted total for game<br>
**pred_total:** Predicted points for team<br>
**opp_total:** Predicted points for opponent<br>
**wind:** Predicted wind speed<br>

In [1]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta
from functions import get_current_weekday, calculate_nfl_week, get_next_sunday, get_current_year

In [2]:
api_key = '7280d6b7b646e8b9467ad216fc7dc0be'

In [3]:
day = get_current_weekday()

In [4]:
date_string = get_next_sunday(day)

In [5]:
week = calculate_nfl_week(date_string)

In [6]:
season = get_current_year()

In [7]:
date_string_monday = (datetime.strptime(date_string, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')

In [8]:
query = requests.get('https://api.the-odds-api.com/v4/sports/americanfootball_nfl/odds/?apiKey=7280d6b7b646e8b9467ad216fc7dc0be&regions=us&markets=spreads,totals&Format=american\
&bookmakers=fanduel,draftkings&dateFormat=iso&commenceTimeFrom=' + date_string + 'T00:00:00Z&commenceTimeTo=' + date_string_monday + 'T23:59:59Z')

In [9]:
json_data = query.json()

In [10]:
odds_df = pd.DataFrame(json_data)

In [11]:
odds_df.tail()

,id,sport_key,sport_title,commence_time,home_team,away_team,bookmakers
9,e61fbdc294fce579705fa0ed6de85ce2,americanfootball_nfl,NFL,2024-09-22T20:25:00Z,Arizona Cardinals,Detroit Lions,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
10,cd904fbb09c5130b857595217364bb07,americanfootball_nfl,NFL,2024-09-22T20:25:00Z,Dallas Cowboys,Baltimore Ravens,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
11,f89d2da2d1b692de14ddf77e186cc5c7,americanfootball_nfl,NFL,2024-09-22T20:25:00Z,Los Angeles Rams,San Francisco 49ers,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
12,bd907bcbaaefa56a30c4fa2c57281a51,americanfootball_nfl,NFL,2024-09-23T00:20:00Z,Atlanta Falcons,Kansas City Chiefs,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
13,96be3ba73fc562e49a0120b5923c3213,americanfootball_nfl,NFL,2024-09-23T23:30:00Z,Buffalo Bills,Jacksonville Jaguars,"[{'key': 'draftkings', 'title': 'DraftKings', ..."


In [12]:
# Function to extract FanDuel and DraftKings spreads and totals
def extract_fanduel_draftkings(row):
    fanduel_away_spread, fanduel_home_spread, fanduel_total = None, None, None
    draftkings_away_spread, draftkings_home_spread, draftkings_total = None, None, None
    
    # Loop through bookmakers to find FanDuel and DraftKings specifically
    for bookmaker in row['bookmakers']:
        if bookmaker['key'] == 'fanduel':
            # Extract spread and total data for FanDuel
            for market in bookmaker['markets']:
                if market['key'] == 'spreads':
                    for outcome in market['outcomes']:
                        if outcome['name'] == row['away_team']:
                            fanduel_away_spread = outcome['point']
                        elif outcome['name'] == row['home_team']:
                            fanduel_home_spread = outcome['point']
                elif market['key'] == 'totals':
                    for outcome in market['outcomes']:
                        if outcome['name'] == 'Over':
                            fanduel_total = outcome['point']
        
        if bookmaker['key'] == 'draftkings':
            # Extract spread and total data for DraftKings
            for market in bookmaker['markets']:
                if market['key'] == 'spreads':
                    for outcome in market['outcomes']:
                        if outcome['name'] == row['away_team']:
                            draftkings_away_spread = outcome['point']
                        elif outcome['name'] == row['home_team']:
                            draftkings_home_spread = outcome['point']
                elif market['key'] == 'totals':
                    for outcome in market['outcomes']:
                        if outcome['name'] == 'Over':
                            draftkings_total = outcome['point']
    
    # Ensure the function returns exactly 6 values, even if some are None
    return pd.Series([fanduel_away_spread, fanduel_home_spread, fanduel_total,
                      draftkings_away_spread, draftkings_home_spread, draftkings_total])

In [13]:
# Apply the function to each row in the DataFrame
odds_df[['fanduel_away_spread', 'fanduel_home_spread', 'fanduel_total',
         'draftkings_away_spread', 'draftkings_home_spread', 'draftkings_total']] = odds_df.apply(extract_fanduel_draftkings, axis=1)# Apply the function to each row in the DataFrame1)

In [14]:
odds_df.drop(columns = ['id', 'sport_key', 'sport_title', 'commence_time', 'bookmakers'], inplace = True)

In [15]:
odds_df

,home_team,away_team,fanduel_away_spread,fanduel_home_spread,fanduel_total,draftkings_away_spread,draftkings_home_spread,draftkings_total
0,Indianapolis Colts,Chicago Bears,1.5,-1.5,43.5,1.0,-1.0,43.5
1,Cleveland Browns,New York Giants,6.5,-6.5,38.5,6.5,-6.5,38.0
2,Tampa Bay Buccaneers,Denver Broncos,6.5,-6.5,40.5,6.0,-6.0,41.0
3,Tennessee Titans,Green Bay Packers,2.5,-2.5,38.5,3.0,-3.0,38.0
4,Minnesota Vikings,Houston Texans,-1.5,1.5,45.5,-1.5,1.5,46.0
5,Pittsburgh Steelers,Los Angeles Chargers,3.0,-3.0,34.5,3.0,-3.0,34.5
6,New Orleans Saints,Philadelphia Eagles,2.5,-2.5,49.5,3.0,-3.0,49.5
7,Las Vegas Raiders,Carolina Panthers,6.5,-6.5,39.5,6.0,-6.0,40.0
8,Seattle Seahawks,Miami Dolphins,4.5,-4.5,42.5,4.5,-4.5,42.0
9,Arizona Cardinals,Detroit Lions,-3.0,3.0,51.5,-3.0,3.0,51.5


In [16]:
# Lists of full team names and their abbreviations
full_names = [
    'Kansas City Chiefs', 'Philadelphia Eagles', 'Buffalo Bills', 'Atlanta Falcons',
    'New Orleans Saints', 'Chicago Bears', 'Cincinnati Bengals', 'Indianapolis Colts',
    'Miami Dolphins', 'New York Giants', 'Seattle Seahawks', 'Los Angeles Chargers',
    'Cleveland Browns', 'Tampa Bay Buccaneers', 'Detroit Lions', 'San Francisco 49ers',
    'Baltimore Ravens', 'Green Bay Packers', 'Arizona Cardinals', 'Pittsburgh Steelers',
    'Carolina Panthers', 'Tennessee Titans', 'New England Patriots', 'Houston Texans',
    'Jacksonville Jaguars', 'Minnesota Vikings', 'Denver Broncos', 'Las Vegas Raiders',
    'Dallas Cowboys', 'Washington Commanders', 'Los Angeles Rams', 'New York Jets'
]

abbreviations = [
    'KC', 'PHI', 'BUF', 'ATL', 'NO', 'CHI', 'CIN', 'IND', 'MIA', 'NYG', 'SEA', 'LAC',
    'CLE', 'TB', 'DET', 'SF', 'BAL', 'GB', 'ARI', 'PIT', 'CAR', 'TEN', 'NE', 'HOU',
    'JAX', 'MIN', 'DEN', 'LV', 'DAL', 'WAS', 'LAR', 'NYJ', 'SF'
]

# Create the dictionary
team_dict = dict(zip(full_names, abbreviations))


In [17]:
# Lists of full team names and their abbreviations
nicknames = [
    'Chiefs', 'Eagles', 'Bills', 'Falcons', 'Saints', 'Bears', 'Bengals', 'Colts',
    'Dolphins', 'Giants', 'Seahawks', 'Chargers', 'Browns', 'Buccaneers', 'Lions', '49ers',
    'Ravens', 'Packers', 'Cardinals', 'Steelers', 'Panthers', 'Titans', 'Patriots', 'Texans',
    'Jaguars', 'Vikings', 'Broncos', 'Raiders', 'Cowboys', 'Commanders', 'Rams', 'Jets'
]

abbreviations = [
    'KC', 'PHI', 'BUF', 'ATL', 'NO', 'CHI', 'CIN', 'IND', 'MIA', 'NYG', 'SEA', 'LAC',
    'CLE', 'TB', 'DET', 'SF', 'BAL', 'GB', 'ARI', 'PIT', 'CAR', 'TEN', 'NE', 'HOU',
    'JAX', 'MIN', 'DEN', 'LV', 'DAL', 'WAS', 'LAR', 'NYJ'
]

# Create the dictionary
mascot_dict = dict(zip(nicknames, abbreviations))

In [18]:
odds_df.replace(team_dict, inplace = True)

In [19]:
odds_df

,home_team,away_team,fanduel_away_spread,fanduel_home_spread,fanduel_total,draftkings_away_spread,draftkings_home_spread,draftkings_total
0,IND,CHI,1.5,-1.5,43.5,1.0,-1.0,43.5
1,CLE,NYG,6.5,-6.5,38.5,6.5,-6.5,38.0
2,TB,DEN,6.5,-6.5,40.5,6.0,-6.0,41.0
3,TEN,GB,2.5,-2.5,38.5,3.0,-3.0,38.0
4,MIN,HOU,-1.5,1.5,45.5,-1.5,1.5,46.0
5,PIT,LAC,3.0,-3.0,34.5,3.0,-3.0,34.5
6,NO,PHI,2.5,-2.5,49.5,3.0,-3.0,49.5
7,LV,CAR,6.5,-6.5,39.5,6.0,-6.0,40.0
8,SEA,MIA,4.5,-4.5,42.5,4.5,-4.5,42.0
9,ARI,DET,-3.0,3.0,51.5,-3.0,3.0,51.5


In [20]:
stadium_dict = {'ATL': {'outdoors': 0, 'grass': 0}, 'ARI': {'outdoors': 0, 'grass': 1}, 'BAL': {'outdoors': 1, 'grass': 1},\
                'BUF': {'outdoors': 1, 'grass': 0}, 'CAR': {'outdoors': 1, 'grass': 0}, 'CHI': {'outdoors': 1, 'grass': 1},\
                'CIN': {'outdoors': 1, 'grass': 0}, 'DAL': {'outdoors': 0, 'grass': 0}, 'DEN': {'outdoors': 1, 'grass': 1},\
                'DET': {'outdoors': 0, 'grass': 0}, 'GB': {'outdoors': 1, 'grass': 1}, 'HOU': {'outdoors': 0, 'grass': 0},\
                'IND': {'outdoors': 0, 'grass': 0}, 'KC': {'outdoors': 1, 'grass': 1}, 'MIA': {'outdoors': 1, 'grass': 1},\
                'JAX': {'outdoors': 1, 'grass': 1}, 'MIN': {'outdoors': 0, 'grass': 1}, 'NE': {'outdoors': 1, 'grass': 0},\
                'NO': {'outdoors': 0, 'grass': 0}, 'NYG': {'outdoors': 1, 'grass': 0}, 'NYJ': {'outdoors': 1, 'grass': 0},\
                'LAC': {'outdoors': 0, 'grass': 0}, 'LAR': {'outdoors': 0, 'grass': 0}, 'SEA': {'outdoors': 1, 'grass': 0},\
                'PIT': {'outdoors': 1, 'grass': 1}, 'CLE': {'outdoors': 1, 'grass': 1}, 'TB': {'outdoors': 1, 'grass': 1},\
                'TEN': {'outdoors': 1, 'grass': 0}, 'WAS': {'outdoors': 1, 'grass': 1}, 'SF': {'outdoors': 1, 'grass': 1},\
                'LV': {'outdoors': 0, 'grass': 0}, 'PHI': {'outdoors': 1, 'grass': 1}}

In [21]:
# Function to get the correct stadium attributes
def add_outdoors_grass(row, stadium_dict):
    home_team = row['home_team']
    return pd.Series(stadium_dict.get(home_team, {'outdoors': None, 'grass': None}))

In [22]:
# Apply the function to add outdoors and grass columns
odds_df[['outdoors', 'grass']] = odds_df.apply(add_outdoors_grass, axis=1, stadium_dict=stadium_dict)

In [23]:
odds_df_fd = odds_df[['home_team', 'away_team', 'fanduel_away_spread', 'fanduel_home_spread', 'fanduel_total', 'outdoors', 'grass']]

In [24]:
odds_df_dk = odds_df[['home_team', 'away_team', 'draftkings_away_spread', 'draftkings_home_spread', 'draftkings_total', 'outdoors', 'grass']]

In [25]:
odds_df_fd_A = odds_df_fd.copy()
odds_df_fd_H = odds_df_fd.copy()
odds_df_dk_A = odds_df_dk.copy()
odds_df_dk_H = odds_df_dk.copy()

In [26]:
odds_df_fd_A.rename(columns = {'away_team': 'team', 'home_team': 'opponent'}, inplace = True)
odds_df_fd_H.rename(columns = {'away_team': 'opponent', 'home_team': 'team'}, inplace = True)
odds_df_dk_A.rename(columns = {'away_team': 'team', 'home_team': 'opponent'}, inplace = True)
odds_df_dk_H.rename(columns = {'away_team': 'opponent', 'home_team': 'team'}, inplace = True)

In [27]:
odds_df_fd_A['home_team'] = 0
odds_df_fd_H['home_team'] = 1
odds_df_dk_A['home_team'] = 0
odds_df_dk_H['home_team'] = 1

In [28]:
odds_df_fd = pd.concat([odds_df_fd_A, odds_df_fd_H])
odds_df_dk = pd.concat([odds_df_dk_A, odds_df_dk_H])

In [29]:
odds_df_fd.head()

,opponent,team,fanduel_away_spread,fanduel_home_spread,fanduel_total,outdoors,grass,home_team
0,IND,CHI,1.5,-1.5,43.5,0,0,0
1,CLE,NYG,6.5,-6.5,38.5,1,1,0
2,TB,DEN,6.5,-6.5,40.5,1,1,0
3,TEN,GB,2.5,-2.5,38.5,1,0,0
4,MIN,HOU,-1.5,1.5,45.5,0,1,0


In [30]:
odds_df_dk.head()

,opponent,team,draftkings_away_spread,draftkings_home_spread,draftkings_total,outdoors,grass,home_team
0,IND,CHI,1.0,-1.0,43.5,0,0,0
1,CLE,NYG,6.5,-6.5,38.0,1,1,0
2,TB,DEN,6.0,-6.0,41.0,1,1,0
3,TEN,GB,3.0,-3.0,38.0,1,0,0
4,MIN,HOU,-1.5,1.5,46.0,0,1,0


In [31]:
odds_df_fd['spread_line'] = np.where(odds_df_fd['home_team'] == 1, odds_df_fd['fanduel_away_spread'], odds_df_fd['fanduel_home_spread'])
odds_df_dk['spread_line'] = np.where(odds_df_dk['home_team'] == 1, odds_df_dk['draftkings_away_spread'], odds_df_dk['draftkings_home_spread'])

In [32]:
odds_df_fd.rename(columns = {'fanduel_total': 'total_line'}, inplace = True)
odds_df_dk.rename(columns = {'draftkings_total': 'total_line'}, inplace = True)

In [33]:
odds_df_fd['pred_total'] = (odds_df_fd['total_line']/2) + (odds_df_fd['spread_line']/2)
odds_df_fd['opp_total'] = (odds_df_fd['total_line']/2) - (odds_df_fd['spread_line']/2)
odds_df_dk['pred_total'] = (odds_df_dk['total_line']/2) + (odds_df_dk['spread_line']/2)
odds_df_dk['opp_total'] = (odds_df_dk['total_line']/2) - (odds_df_dk['spread_line']/2)

In [34]:
odds_df_fd.drop(columns = ['fanduel_away_spread', 'fanduel_home_spread'], inplace = True)
odds_df_dk.drop(columns = ['draftkings_away_spread', 'draftkings_home_spread'], inplace = True)

In [35]:
odds_df_fd.head()

,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total
0,IND,CHI,43.5,0,0,0,-1.5,21.0,22.5
1,CLE,NYG,38.5,1,1,0,-6.5,16.0,22.5
2,TB,DEN,40.5,1,1,0,-6.5,17.0,23.5
3,TEN,GB,38.5,1,0,0,-2.5,18.0,20.5
4,MIN,HOU,45.5,0,1,0,1.5,23.5,22.0


In [36]:
odds_df_dk.head()

,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total
0,IND,CHI,43.5,0,0,0,-1.0,21.25,22.25
1,CLE,NYG,38.0,1,1,0,-6.5,15.75,22.25
2,TB,DEN,41.0,1,1,0,-6.0,17.50,23.50
3,TEN,GB,38.0,1,0,0,-3.0,17.50,20.50
4,MIN,HOU,46.0,0,1,0,1.5,23.75,22.25


In [37]:
from bs4 import BeautifulSoup

In [38]:
url = "https://rotogrinders.com/weather/nfl"

In [39]:
response = requests.get(url)

In [40]:
soup = BeautifulSoup(response.text, 'html.parser')

In [41]:
#This code inititializes two lists and a dict.
#Then we get all the team names from the soup that we scraped.
#Then we get all the weather data
#To get the wind speed, we take every fourth item of the wind_mph list, because the other stuff is temps and such
#To get home homes, we take every second team name.
#Then we have a stadium dict where every value is another dict with keys outdoors and grass
#If the home team plays outdoors, we take the next element in the wind_speeds list.
#But here's the question. The lengths don't match. If a team doesn't play outdoors, the wind speed will be 0
#We don't need to go into the list for that because there won't be a corresponding element.
#So how do I just get the 'next' item in the wind_speeds list?

home_teams = []
wind_speeds = []
wind_dict = {}

# Find all game containers
team_mascots = soup.find_all('span', class_='team-nameplate-mascot')
wind_mph = soup.find_all('span', class_='weather-gametime-value bold')

for mph in wind_mph[3::4]:
    wind_speeds.append(mph.get_text())

for mascot in team_mascots[1::2]:
    mascot_abbr = mascot_dict[mascot.get_text()]
    home_teams.append(mascot_abbr)

wind_iterator = iter(wind_speeds)  # Create an iterator for wind speeds
for team in home_teams:
    if stadium_dict[team]['outdoors'] == 1:
        wind_dict[team] = next(wind_iterator)
    else:
        wind_dict[team] = 0
        
#     home_teams.append

# for mph in wind_mph:
#     print(mph.get_text())
# print(team_mascots)
# print(wind_mph)

# for game in game_containers:
#     # Extract teams
#     teams = game.find_all('span', class_='team-nameplate-title')
#     team_1 = f"{teams[0].find('span', class_='team-nameplate-city').text} {teams[0].find('span', class_='team-nameplate-mascot').text}"
#     team_2 = f"{teams[1].find('span', class_='team-nameplate-city').text} {teams[1].find('span', class_='team-nameplate-mascot').text}"
    
#     # Extract game time
#     game_time = game.find('span', class_='game-weather-time').text.strip().split(' ET')[0]
    
#     # Extract wind speed
#     wind_speed_value = game.find('span', class_='weather-gametime-value bold').text.strip()
#     wind_speed_unit = game.find('span', class_='weather-gametime-label muted').text.strip()

#     # Combine wind speed with unit (e.g., '11 mph')
#     wind_speed = f"{wind_speed_value} {wind_speed_unit}"
    
#     # Check if the game is in a dome
#     is_dome = bool(game.find('div', class_='weather-column-empty'))
    
#     # Append game info to the list
#     games.append({
#         'Teams': f"{team_1} vs {team_2}",
#         'Game Time': game_time,
#         'Wind Speed': wind_speed,
#         'Is Dome': is_dome
#     })

# # Print the results
# for game in games:
#     print(game)
wind_dict

{'NYJ': '5',
 'TB': '5',
 'NO': 0,
 'TEN': '4',
 'IND': 0,
 'PIT': '2',
 'MIN': 0,
 'CLE': '5',
 'LV': 0,
 'SEA': '4',
 'LAR': 0,
 'DAL': 0,
 'ARI': 0,
 'ATL': 0,
 'BUF': '6',
 'CIN': '3'}

In [42]:
#<span class="team-nameplate-title" data-abbr="NEP"> <span class="team-nameplate-city">New England</span> <span class="team-nameplate-mascot">Patriots</span> </span>

In [43]:
#<span class="game-weather-time">9/19 8:15 PM ET <span class="gray game-weather-stadium">AT METLIFE STADIUM</span></span>

In [44]:
#<td class="muted lightgray"><div class="weather-column-wind"><span>N</span><span>11 mph</span></div></td>
#<div class="weather-column-wind"><span>N</span><span>11 mph</span></div>

In [45]:
#Wind
#We should scrape this from somewhere. For now it's a static list
#wind_dict = {'GB': 5, 'TEN': 8, 'NE': 6, 'WAS': 8, 'CAR': 14, 'BAL': 12, 'JAX': 10, 'DEN': 6, 'KC': 9}

In [46]:
def add_wind(row, wind_dict):
    if row['home_team'] == 1:  # Home team case
        return wind_dict.get(row['team'], 0)  # Default to 0 if not found
    else:  # Away team case
        return wind_dict.get(row['opponent'], 0)  # Default to 0 if not found

In [47]:
odds_df_fd['wind'] = odds_df_fd.apply(add_wind, axis=1, wind_dict=wind_dict)
odds_df_dk['wind'] = odds_df_dk.apply(add_wind, axis=1, wind_dict=wind_dict)

In [48]:
odds_df_fd['season'] = 2024
odds_df_dk['season'] = 2024
odds_df_fd['week'] = week
odds_df_dk['week'] = week

In [49]:
odds_df_fd.head()

,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total,wind,season,week
0,IND,CHI,43.5,0,0,0,-1.5,21.0,22.5,0,2024,3
1,CLE,NYG,38.5,1,1,0,-6.5,16.0,22.5,5,2024,3
2,TB,DEN,40.5,1,1,0,-6.5,17.0,23.5,5,2024,3
3,TEN,GB,38.5,1,0,0,-2.5,18.0,20.5,4,2024,3
4,MIN,HOU,45.5,0,1,0,1.5,23.5,22.0,0,2024,3


In [50]:
odds_df_fd.tail()

,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total,wind,season,week
9,DET,ARI,51.5,0,1,1,-3.0,24.25,27.25,0,2024,3
10,BAL,DAL,47.5,0,0,1,-1.5,23.00,24.50,0,2024,3
11,SF,LAR,43.5,0,0,1,-6.5,18.50,25.00,0,2024,3
12,KC,ATL,46.5,0,0,1,-3.0,21.75,24.75,0,2024,3
13,JAX,BUF,45.5,1,0,1,4.5,25.00,20.50,6,2024,3


In [51]:
odds_df_dk.head()

,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total,wind,season,week
0,IND,CHI,43.5,0,0,0,-1.0,21.25,22.25,0,2024,3
1,CLE,NYG,38.0,1,1,0,-6.5,15.75,22.25,5,2024,3
2,TB,DEN,41.0,1,1,0,-6.0,17.50,23.50,5,2024,3
3,TEN,GB,38.0,1,0,0,-3.0,17.50,20.50,4,2024,3
4,MIN,HOU,46.0,0,1,0,1.5,23.75,22.25,0,2024,3


In [52]:
odds_df_dk.tail()

,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total,wind,season,week
9,DET,ARI,51.5,0,1,1,-3.0,24.25,27.25,0,2024,3
10,BAL,DAL,47.5,0,0,1,-1.0,23.25,24.25,0,2024,3
11,SF,LAR,43.0,0,0,1,-6.5,18.25,24.75,0,2024,3
12,KC,ATL,46.5,0,0,1,-3.0,21.75,24.75,0,2024,3
13,JAX,BUF,45.5,1,0,1,5.0,25.25,20.25,6,2024,3


In [53]:
odds_df_fd.to_csv('fd_spreads_' + str(season) + '_' + str(week) + '.csv')
odds_df_dk.to_csv('dk_spreads_' + str(season) + '_' + str(week) + '.csv')